In [7]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
text='''Prime Minister Narendra Modi launched the Ayushman Bharat Health Infrastructure Mission, one of the largest pan-India schemes for strengthening healthcare infrastructure, in his parliamentary constituency Varanasi in Uttar Pradesh on Monday.

The Prime Minister also inaugurated various development projects worth more than ₹5,200 crore for his constituency.

The Pradhan Mantri Ayushman Bharat Health Infrastructure Mission is one of the largest pan-India schemes for strengthening healthcare infrastructure across the country. It is in addition to the National Health Mission.

Its objective is to fill gaps in public health infrastructure, especially in critical care facilities and primary care in both urban and rural areas. It will provide support for 17,788 rural health and wellness centres in 10 high-focus states. Further, 11,024 urban health and wellness centres will be established in all the States.


Through this, critical care services will be available in all the districts of the country with more than five lakh population through exclusive critical care hospital blocks, while the remaining districts will be covered through referral services.


People will have access to a full range of diagnostic services in the public healthcare system through a network of laboratories across the country, and integrated public health labs will be set up in all the districts.

Under the scheme, a national institution for one health, four new national institutes for virology, a regional research platform for WHO South East Asia Region, nine biosafety level-III laboratories, five new regional national centre for disease control will be set up.'''

In [14]:
sentences=sent_tokenize(text)

In [15]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

In [16]:
w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [17]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

In [18]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [19]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
for sent in sentences:
    if sent in top.keys():
        print(sent)

The Pradhan Mantri Ayushman Bharat Health Infrastructure Mission is one of the largest pan-India schemes for strengthening healthcare infrastructure across the country.
Its objective is to fill gaps in public health infrastructure, especially in critical care facilities and primary care in both urban and rural areas.
Through this, critical care services will be available in all the districts of the country with more than five lakh population through exclusive critical care hospital blocks, while the remaining districts will be covered through referral services.
People will have access to a full range of diagnostic services in the public healthcare system through a network of laboratories across the country, and integrated public health labs will be set up in all the districts.
